# COMP 472 - Assignment 2

### Bernard Claveau - 40065756 / Nicolas Eliopoulos - 40059378

## Contents

- [Imports](#Imports)
    
- [Puzzle Search](#Puzzle-Search)
    - [State representation](#State-representation)
    - [Successor function](#Successor-function)
    - [Goal function](#Goal-function)
    - [Solution path function](#Solution-path-function)
    
- [Heuristics](#Heuristics)
    - [Heuristic 1 - Hamming Distance](#Heuristic-1---h1)
    - [Heuristic 2 - Sum of Lowest Costs](#Heuristic-2---h2)
    - [Heuristic 0](#Heuristic-0---h0-(for-demo))
    
- [Uniform-Cost Search (UCS)](#Uniform-Cost-Search-(UCS))
    - [State representation](#State-representation-(UCS))
    - [Successor function](#Successor-function-(UCS))
    - [Algorithm](#Algorithm-(UCS))
    
- [Greedy Best-First Search (GBFS)](#Greedy-Best-First-Search-(GBFS))
    - [State representation](#State-representation-(GBFS))
    - [Successor function](#Successor-function-(GBFS))
    - [Algorithm](#Algorithm-(GBFS))
    
- [Algorithm A*](#Algorithm-A*)
    - [State representation](#State-representation-(A*))
    - [Successor function](#Successor-function-(A*))
    - [Algorithm](#Algorithm-(A*))
    
- [Functions](#Functions)
    - [Write output to file](#Write-output-to-file)
    - [Read puzzles from input file](#Read-puzzles-from-input-file)
    - [Solve puzzle set](#Solve-puzzle-set)
    - [Analyze search algorithm](#Analyze-search-algorithm)
    - [Generate random puzzle set](#Generate-random-puzzle-set)
    
- [Output](#Output)
    
- [Analysis](#Analysis)

- [Scaling Up](#Scaling-Up)

- [Archive](#Archive)
    

## Imports

In [1]:
from functools import total_ordering
from heapq import heapify, heappop, heappush
import time
import random

## Puzzle Search

This section contains the implementation of classes and functions required for solving the X-puzzle and that are common to all three search algorithms.

### State representation

This is the base node class that is extended by each algorithm implementation.

In [2]:
class Node:
    def __init__(self, state, parent=None, move=0, cost=0, tile=0, seq=0):
        self.state = state
        self.parent = parent
        self.move = move
        self.cost = cost
        self.seq = seq
        self.tile = tile
        
        
def tostr(state):
    return ' '.join(map(str, state))

### Successor function

This function returns the list of successor nodes for a given node.

In [3]:
def successor(node, m, n):
    state = node.state
    i = state.index(0)
    successors = []

    # regular move: right
    if (i + 1) % n != 0:
        temp = state.copy()
        temp[i], temp[i + 1] = temp[i + 1], temp[i]
        successors.append(Node(temp, move=1, tile=temp[i]))
    
    # regular move: left
    if i % n != 0:
        temp = state.copy()
        temp[i], temp[i - 1] = temp[i - 1], temp[i]
        successors.append(Node(temp, move=1, tile=temp[i]))

    # regular move: up
    if i >= n:
        temp = state.copy()
        temp[i], temp[i - n] = temp[i - n], temp[i]
        successors.append(Node(temp, move=1, tile=temp[i]))

    # regular move: down
    if i < m * n - n:
        temp = state.copy()
        temp[i], temp[i + n] = temp[i + n], temp[i]
        successors.append(Node(temp, move=1, tile=temp[i]))

    # wrap move: right
    if (i + 1) % n == 0 and n > 2:
        temp = state.copy()
        temp[i], temp[i - n + 1] = temp[i - n + 1], temp[i]
        successors.append(Node(temp, move=2, tile=temp[i]))
        
    # wrap move: left
    if i % n == 0 and n > 2:
        temp = state.copy()
        temp[i], temp[i + n - 1] = temp[i + n - 1], temp[i]
        successors.append(Node(temp, move=2, tile=temp[i]))
        
    # wrap move: up
    if i < n and m > 2:
        temp = state.copy()
        temp[i], temp[i + n*(m - 1)] = temp[i + n*(m - 1)], temp[i]
        successors.append(Node(temp, move=2, tile=temp[i]))
    
    # wrap move: down
    if i >= m * n - n and m > 2:
        temp = state.copy()
        temp[i], temp[i - n*(m - 1)] = temp[i - n*(m - 1)], temp[i]
        successors.append(Node(temp, move=2, tile=temp[i]))
    
    # diagonal move: top-left adjacent
    if i == 0:
        temp = state.copy()
        temp[i], temp[n + 1] = temp[n + 1], temp[i]
        successors.append(Node(temp, move=3, tile=temp[i]))
    
    # diagonal move: top-right adjacent
    if i == n - 1:
        temp = state.copy()
        temp[i], temp[2*i] = temp[2*i], temp[i]
        successors.append(Node(temp, move=3, tile=temp[i]))

    # diagonal move: bottom-left adjacent
    if i == m * n - n:
        temp = state.copy()
        temp[i], temp[i - n + 1] = temp[i - n + 1], temp[i]
        successors.append(Node(temp, move=3, tile=temp[i]))
        
    # diagonal move: bottom-right adjacent
    if i == m * n - 1:
        temp = state.copy()
        temp[i], temp[i - n - 1] = temp[i - n - 1], temp[i]
        successors.append(Node(temp, move=3, tile=temp[i]))

    # diagonal move: opposite
    if (i == 0) or (i == n - 1) or (i == m * n - n) or (i == m * n - 1):
        temp = state.copy()
        temp[i], temp[n * m - i - 1] = temp[n * m - i - 1], temp[i]
        successors.append(Node(temp, move=3, tile=temp[i]))

    return successors

### Goal function

This function returns True if the given state matches one of the two possible goal states.

In [4]:
def goal(state, m, n):
    return state == goal1(m, n) or state == goal2(m, n)

In [5]:
def goal1(m, n):
    gs = []
    
    for i in range(1, m*n):
        gs.append(i)
    gs.append(0)
    
    return gs

In [6]:
def goal2(m, n):
    gs = [0] * m*n
    tile = 1
    
    for i in range(n):
        for j in range(m):
            gs[j*n + i] = tile if tile < m*n else 0
            tile += 1
    
    return gs

### Solution path function

This function traces the path from a goal state to the start state and returns the solution path.

In [7]:
def solution_path(node):
    current = node
    sol_path = [current]
    while current.parent:
        sol_path.append(current.parent)
        current = current.parent
    sol_path.reverse()
    return sol_path

## Heuristics

We have chosen the following heuristics h<sub>1</sub> and h<sub>2</sub> to use for the Greedy Best-First Search and A* algorithms.<br>

Heuristic h<sub>0</sub> is to be used for the demo input as described in the assignment handout.

### Heuristic 1 - h<sub>1</sub>

#### Hamming Distance

The Hamming distance is the number of tiles that are out of place. 

The heuristic funtion calculates this value for both versions of the goal state and returns the lower one.

In [8]:
def h1(state, m, n):
    hd1 = 0
    hd2 = 0
    
    for i in range(m*n - 1):
        if state[i] != i + 1:
            hd1 += 1
       
    tile = 1
    for i in range(n):
        for j in range(m):
            if j*n + i != m*n - 1 and state[j*n + i] != tile:
                hd2 += 1
            tile += 1
        
    return min(hd1, hd2)

### Heuristic 2 - h<sub>2</sub>

#### Sum of Lowest Costs

This heuristic is inspired by the Manhattan distance but takes into account **wrapping moves** and **diagonal moves**.

For each tile, it calculates the lowest cost required to move the tile from its actual position to its goal position if there were no other tiles in the way. It then takes the sum of this cost for all the tiles.

Again, the heuristic function calculates the sum of lowest costs for both versions of the goal state and returns the lower one.

In [9]:
def h2(state, m, n):
    md1 = 0
    md2 = 0
    
    for i in range(1, m*n):
        current = state.index(i)
        goal1 = i - 1
        goal2 = n * ((i - 1) % m) + (i - 1) // m
        
        # horizontal distance
        hd1 = abs(current % n - goal1 % n)
        hd2 = abs(current % n - goal2 % n)
        hw1 = n - hd1 + 1 
        hw2 = n - hd2 + 1
        
        md1 += min(hd1, hw1)
        md2 += min(hd2, hw2)
        
        # vertical distance
        vd1 = abs(((current - current % n) - (goal1 - goal1 % n)) // n)
        vd2 = abs(((current - current % n) - (goal2 - goal2 % n)) // n)
        vw1 = m - vd1 + 1
        vw2 = m - vd2 + 1
        
        md1 += min(vd1, vw1)
        md2 += min(vd2, vw2)
        
        # decrement the cost if a diagonal can be taken
        if hw1 <= hd1 and vw1 <= vd1:
            md1 -= 1
            
        if hw2 <= hd2 and vw2 <= vd2:
            md2 -= 1
        
    return min(md1, md2)

### Heuristic 0 - h<sub>0</sub> (for demo)

The heuristic h<sub>0</sub> is given in the assignment handout and returns 0 for any state with the empty tile (0) in the bottom right corner, and 1 otherwise.

In [10]:
def h0(state, m, n):
    if(state.index(0) == m*n - 1):
        return 0
    else:
        return 1

## Uniform-Cost Search (UCS)

### State representation (UCS)

In [11]:
@total_ordering
class UCSNode(Node):
    
    def __init__(self, state, parent=None, move=0, cost=0, tile=0, seq=0):
        Node.__init__(self, state, parent, move, cost, tile, seq)

    def __lt__(self, other):
        return self.cost < other.cost or (self.cost == other.cost and self.seq < other.seq)

### Successor function (UCS)

In [12]:
def ucs_successor(node, m, n, seq):
    successors = successor(node, m, n)
    ucs_nodes = []
    for child in successors:
        ucs_nodes.append(UCSNode(child.state, node, child.move, node.cost + child.move, child.tile, seq))
        seq += 1
    return ucs_nodes

### Algorithm (UCS)

In [13]:
def ucs(puzzle, m, n, timeout=True):
    start = time.time()
    opened = []
    closed = []
    open_set = set()
    closed_set = set()
    seq = 0
    
    heappush(opened, UCSNode(puzzle, seq=seq))
    seq += 1
    open_set.add(tostr(puzzle))

    while opened:
        if timeout and time.time() - start > 60:
            print('UCS timed out after 60 seconds on puzzle', puzzle)
            return None, closed, 60
        
        current = heappop(opened)
        while current.state == -1:
            current = heappop(opened)
            
        closed.append(current)
        
        open_set.remove(tostr(current.state))
        closed_set.add(tostr(current.state))
        
        if goal(current.state, m, n):
            t = round(time.time() - start, 5)
            print('UCS found solution for', puzzle, 'with cost', current.cost, 'in', t, 'seconds')
            return solution_path(current), closed, t
        
        children = ucs_successor(current, m, n, seq)
        for i in children:
            seq += 1
            i_state = tostr(i.state)
            
            if i_state in closed_set:
                continue

            if i_state in open_set:
                existing = next(x for x in opened if x.state == i.state)
                if i < existing:
                    existing.state = -1
                    heappush(opened, i)
                continue
    
            heappush(opened, i)
            open_set.add(i_state)
                
    t = round(time.time() - start, 5)
    print('UCS exhausted every path in', t, 'seconds without finding a solution')
    return None, closed, t

## Greedy Best-First Search (GBFS)

### State representation (GBFS)

In [14]:
@total_ordering
class GBFSNode:
    def __init__(self, state, parent=None, move=0, cost=0, h=0, tile=0, seq=0):
        Node.__init__(self, state, parent, move, cost, tile, seq)
        self.h = h

    def __lt__(self, other):
        return self.h < other.h or (self.h == other.h and self.seq < other.seq)
    

### Successor function (GBFS)

In [15]:
def gbfs_successor(node, m, n, seq, heuristic):
    successors = successor(node, m, n)
    gbfs_nodes = []
    for child in successors:
        h = heuristic(child.state, m, n)
        gbfs_nodes.append(GBFSNode(child.state, node, child.move, node.cost + child.move, h, child.tile, seq))
        seq += 1
    return gbfs_nodes

### Algorithm (GBFS)

In [16]:
def gbfs(puzzle, m , n, heuristic, timeout=True):
    start = time.time()
    opened = []
    closed = []
    open_set = set()
    closed_set = set()
    seq = 0
    
    heappush(opened, GBFSNode(puzzle, h=heuristic(puzzle, m, n), seq=seq))
    seq += 1
    open_set.add(tostr(puzzle))

    while opened:
        if timeout and time.time() - start > 60:
            print('GBFS', heuristic.__name__, 'timed out after 60 seconds on puzzle', puzzle)
            return None, closed, 60
        
        current = heappop(opened)
        closed.append(current)
        
        open_set.remove(tostr(current.state))
        closed_set.add(tostr(current.state))
        
        if goal(current.state, m, n):
            t = round(time.time() - start, 5)
            print('GBFS', heuristic.__name__, 'found solution for', puzzle, 'with cost', current.cost, 'in', t, 'seconds')
            return solution_path(current), closed, t
        
        children = gbfs_successor(current, m, n, seq, heuristic)
        for i in children:
            seq += 1
            i_state = tostr(i.state)  
            
            if i_state in open_set or i_state in closed_set:
                continue
            
            heappush(opened, i)
            open_set.add(i_state)
                
    t = round(time.time() - start, 5)
    print('GBFS exhausted every path in', t, 'seconds without finding a solution')
    return None, closed, t

## Algorithm A*

### State representation (A*)

In [17]:
@total_ordering
class ANode:
    def __init__(self, state, parent=None, move=0, cost=0, h=0, tile=0, seq=0):
        Node.__init__(self, state, parent, move, cost, tile, seq)
        self.h = h

    def __lt__(self, other):
        return self.h + self.cost < other.h + other.cost or (
                    self.h + self.cost == other.h + other.cost and self.seq < other.seq)

### Successor function (A*)

In [18]:
def a_star_successor(node, m, n, seq, heuristic):
    successors = successor(node, m, n)
    a_nodes = []
    for child in successors:
        h = heuristic(child.state, m, n)
        a_nodes.append(ANode(child.state, node, child.move, node.cost + child.move, h, child.tile, seq))
        seq += 1
    return a_nodes

### Algorithm (A*)

In [63]:
def a_star(puzzle, m, n, heuristic, timeout=True):
    start = time.time()
    opened = []
    closed = []
    open_set = set()
    closed_set = set()
    seq = 0
    
    heappush(opened, ANode(puzzle, h=heuristic(puzzle, m, n), seq=seq))
    seq += 1
    open_set.add(tostr(puzzle))

    while opened:
        if timeout and time.time() - start > 60:
            print('A*', heuristic.__name__, ' timed out after 60 seconds on puzzle', puzzle)
            return None, closed, 60
        
        current = heappop(opened)
        while current.state == -1:
            current = heappop(opened)
            
        closed.append(current)
        
        open_set.remove(tostr(current.state))
        closed_set.add(tostr(current.state))
        
        if goal(current.state, m, n):
            t = round(time.time() - start, 5)
            print('A*', heuristic.__name__, 'found solution for', puzzle, 'with cost', current.cost, 'in', t, 'seconds')
            return solution_path(current), closed, t
        
        children = a_star_successor(current, m, n, seq, heuristic)
        for i in children:
            seq += 1
            i_state = tostr(i.state)

            if i_state in closed_set:
                existing = next(x for x in closed if x.state == i.state and x.seq != -1)
                if i < existing:
                    existing.seq = -1
                    closed_set.remove(tostr(existing.state))
                    heappush(opened, i)
                    open_set.add(i_state)
                continue

            if i_state in open_set:
                existing = next(x for x in opened if x.state == i.state)
                if i < existing:
                    existing.state = -1
                    heappush(opened, i)
                continue
    
            heappush(opened, i)
            open_set.add(i_state)
                
    t = round(time.time() - start, 5)
    print('A* exhausted every path in', t, 'seconds without finding a solution')
    return None, closed, t

## Functions

This section contains the important functions used in the [Output](#Output), [Analysis](#Analysis), and [Scaling Up](#Scaling-Up) sections.

#### Write output to file

These functions write the output of a search algorithm to a file.

In [20]:
def write_sol_path(file_path, sol_path, time):
    with open(file_path, 'w') as f:
        if not sol_path:
            f.write('no solution')
        else:
            for node in sol_path:
                f.write(str(node.tile) + ' ' + str(node.move))
                for i in node.state:
                    f.write(' ' + str(i))
                f.write('\n')
            f.write(str(sol_path[-1].cost) + ' ' + str(time))    

In [21]:
def write_search_path_ucs(file_path, search_path, sol):
    with open(file_path, 'w') as f:
        if not sol:
            f.write('no solution')
        else:
            for node in search_path:
                f.write('0 ' + str(node.cost) + ' 0')
                for i in node.state:
                    f.write(' ' + str(i))
                f.write('\n')

In [22]:
def write_search_path_gbfs(file_path, search_path, sol):
    with open(file_path, 'w') as f:
        if not sol:
            f.write('no solution')
        else:
            for node in search_path:
                f.write('0 0 ' + str(node.h))
                for i in node.state:
                    f.write(' ' + str(i))
                f.write('\n')

In [23]:
def write_search_path_astar(file_path, search_path, sol):
    with open(file_path, 'w') as f:
        if not sol:
            f.write('no solution')
        else:
            for node in search_path:
                f.write(str(node.cost + node.h) + ' ' + str(node.cost) + ' ' + str(node.h))
                for i in node.state:
                    f.write(' ' + str(i))
                f.write('\n')  

#### Read puzzles from input file

This function reads an input file of puzzles and returns them in the format used by the algorithms.

In [24]:
def get_puzzle_set(file_name):
    puzzle_set = []

    with open(file_name, 'r') as sample_puzzles:
        line = sample_puzzles.readline()
        while line:
            puzzle_set.append([int(x) for x in line.split()])
            line = sample_puzzles.readline()
    
    return puzzle_set

#### Solve puzzle set

This function runs the five search aglorithms on all puzzles in a puzzle set and writes the results to corresponding output files.

In [25]:
def solve_puzzle_set(puzzle_set, m, n, folder):   
    for i, puzzle in enumerate(puzzle_set):
        num = str(i)

        # UCS
        sol, search, t = ucs(puzzle, m, n)

        write_sol_path(folder + '/' + num + '_ucs_solution.txt', sol, t)
        write_search_path_ucs(folder + '/' + num + '_ucs_search.txt', search, sol)

        # GBFS with h1
        sol, search, t = gbfs(puzzle, m, n, h1)

        write_sol_path(folder + '/' + num + '_gbfs-h1_solution.txt', sol, t)
        write_search_path_gbfs(folder + '/' + num + '_gbfs-h1_search.txt', search, sol)

        # GBFS with h2
        sol, search, t = gbfs(puzzle, m, n, h2)

        write_sol_path(folder + '/' + num + '_gbfs-h2_solution.txt', sol, t)
        write_search_path_gbfs(folder + '/' + num + '_gbfs-h2_search.txt', search, sol)

        # A* with h1
        sol, search, t = a_star(puzzle, m, n, h1)

        write_sol_path(folder + '/' + num + '_astar-h1_solution.txt', sol, t)
        write_search_path_astar(folder + '/' + num + '_astar-h1_search.txt', search, sol)

        # A* with h2
        sol, search, t = a_star(puzzle, m, n, h2)

        write_sol_path(folder + '/' + num + '_astar-h2_solution.txt', sol, t)
        write_search_path_astar(folder + '/' + num + '_astar-h2_search.txt', search, sol)

#### Analyze search algorithm

This function runs a search algorithm on a puzzle set and then computes and displays a set of metrics.

In [36]:
def analyze(algorithm, puzzle_set, m, n, h=None, timeout=True):
    sol_len = 0
    search_len = 0
    cost = 0
    no_sol = 0
    exec_time = 0
    
    for puzzle in puzzle_set:
        sol, search, t = algorithm(puzzle, m, n, h, timeout=timeout) if h else algorithm(puzzle, m, n, timeout=timeout)
        
        if sol:
            exec_time += t
            sol_len += len(sol)
            search_len += len(search)
            cost += sol[-1].cost
        else:
            no_sol += 1
    
    solves = len(puzzle_set) - no_sol
    
    print('ANALYSIS OF', algorithm.__name__.upper(), 'WITH ' + h.__name__.upper() if h else '')
    print()
    print('Average length of solution path:', round(sol_len / solves, 2))
    print('Total length of solution path:', sol_len)
    print()
    print('Average length of search path:', round(search_len / solves, 2))
    print('Total length of search path:', search_len)
    print()
    print('Average cost:', round(cost / solves, 2))
    print('Total cost:', cost)
    print()
    print('Rate of no solution:', round(no_sol / len(puzzle_set), 3))
    print('Total number of no solution', no_sol)
    print()
    print('Average execution time:', round(exec_time / solves, 5))
    print('Total execution time:', exec_time)

#### Generate random puzzle set

This function generates a random set of puzzles.

In [27]:
def random_puzzle_set(m, n, num, file_path=None):
    gs1 = goal1(m, n)
    gs2 = goal2(m, n)
    puzzle_set = []
    
    while len(puzzle_set) < num // 2:
        p = gs1.copy()
        random.shuffle(p)
        if p != gs1 and p != gs2 and p not in puzzle_set:
            puzzle_set.append(p)
        
    while len(puzzle_set) < num:
        p = gs2.copy()
        random.shuffle(p)
        if p != gs1 and p != gs2 and p not in puzzle_set:
            puzzle_set.append(p)

    if file_path:
        with open(file_path, 'w') as f:
            for puzzle in puzzle_set:
                f.write(tostr(puzzle) + '\n')
            
    return puzzle_set

## Output

In [34]:
# Read input file from moodle
input_set = get_puzzle_set('samplePuzzles.txt')

# Sovle puzzle set and write output to files
solve_puzzle_set(input_set, 2, 4, )

UCS found solution for [3, 0, 1, 4, 2, 6, 5, 7] with cost 13 in 9.22443 seconds
GBFS h1 found solution for [3, 0, 1, 4, 2, 6, 5, 7] with cost 17 in 0.00299 seconds
GBFS h2 found solution for [3, 0, 1, 4, 2, 6, 5, 7] with cost 24 in 0.01595 seconds
A* h1 found solution for [3, 0, 1, 4, 2, 6, 5, 7] with cost 13 in 0.07822 seconds
A* h2 found solution for [3, 0, 1, 4, 2, 6, 5, 7] with cost 13 in 0.02294 seconds
UCS found solution for [6, 3, 4, 7, 1, 2, 5, 0] with cost 12 in 8.80787 seconds
GBFS h1 found solution for [6, 3, 4, 7, 1, 2, 5, 0] with cost 40 in 0.00399 seconds
GBFS h2 found solution for [6, 3, 4, 7, 1, 2, 5, 0] with cost 18 in 0.00499 seconds
A* h1 found solution for [6, 3, 4, 7, 1, 2, 5, 0] with cost 12 in 0.02593 seconds
A* h2 found solution for [6, 3, 4, 7, 1, 2, 5, 0] with cost 12 in 0.00997 seconds
UCS found solution for [1, 0, 3, 6, 5, 2, 7, 4] with cost 9 in 0.20941 seconds
GBFS h1 found solution for [1, 0, 3, 6, 5, 2, 7, 4] with cost 17 in 0.00399 seconds
GBFS h2 found

## Analysis

In [35]:
# Generate a puzzle set of 50 random puzzles (2x4)
random_puzzle_set(2, 4, 50, 'analysis/analysis_set_2x4.txt')

analysis_set = get_puzzle_set('analysis/analysis_set_2x4.txt')

In [37]:
analyze(ucs, analysis_set, 2, 4)

UCS found solution for [2, 0, 5, 6, 1, 3, 4, 7] with cost 13 in 9.7876 seconds
UCS found solution for [0, 2, 1, 4, 3, 6, 5, 7] with cost 14 in 22.20186 seconds
UCS found solution for [4, 6, 5, 2, 1, 3, 7, 0] with cost 14 in 31.1036 seconds
UCS found solution for [6, 4, 3, 2, 1, 7, 0, 5] with cost 13 in 14.91904 seconds
UCS found solution for [6, 7, 3, 2, 0, 1, 5, 4] with cost 13 in 7.61595 seconds
UCS found solution for [1, 3, 6, 4, 5, 7, 2, 0] with cost 14 in 32.40468 seconds
UCS found solution for [7, 6, 0, 1, 2, 4, 5, 3] with cost 15 in 53.98124 seconds
UCS timed out after 60 seconds on puzzle [7, 5, 3, 2, 1, 0, 4, 6]
UCS found solution for [2, 0, 1, 4, 7, 5, 3, 6] with cost 13 in 9.78554 seconds
UCS timed out after 60 seconds on puzzle [6, 4, 3, 1, 7, 5, 0, 2]
UCS found solution for [5, 4, 0, 3, 1, 2, 6, 7] with cost 9 in 0.07081 seconds
UCS found solution for [0, 2, 1, 5, 3, 4, 7, 6] with cost 13 in 13.53286 seconds
UCS found solution for [2, 4, 5, 3, 7, 1, 6, 0] with cost 14 in 1

In [38]:
analyze(gbfs, analysis_set, 2, 4, h1)

GBFS h1 found solution for [2, 0, 5, 6, 1, 3, 4, 7] with cost 14 in 0.001 seconds
GBFS h1 found solution for [0, 2, 1, 4, 3, 6, 5, 7] with cost 29 in 0.00698 seconds
GBFS h1 found solution for [4, 6, 5, 2, 1, 3, 7, 0] with cost 30 in 0.00798 seconds
GBFS h1 found solution for [6, 4, 3, 2, 1, 7, 0, 5] with cost 31 in 0.00402 seconds
GBFS h1 found solution for [6, 7, 3, 2, 0, 1, 5, 4] with cost 35 in 0.00702 seconds
GBFS h1 found solution for [1, 3, 6, 4, 5, 7, 2, 0] with cost 30 in 0.01097 seconds
GBFS h1 found solution for [7, 6, 0, 1, 2, 4, 5, 3] with cost 41 in 0.00798 seconds
GBFS h1 found solution for [7, 5, 3, 2, 1, 0, 4, 6] with cost 25 in 0.00399 seconds
GBFS h1 found solution for [2, 0, 1, 4, 7, 5, 3, 6] with cost 27 in 0.001 seconds
GBFS h1 found solution for [6, 4, 3, 1, 7, 5, 0, 2] with cost 33 in 0.00299 seconds
GBFS h1 found solution for [5, 4, 0, 3, 1, 2, 6, 7] with cost 18 in 0.002 seconds
GBFS h1 found solution for [0, 2, 1, 5, 3, 4, 7, 6] with cost 36 in 0.00399 second

In [39]:
analyze(gbfs, analysis_set, 2, 4, h2)

GBFS h2 found solution for [2, 0, 5, 6, 1, 3, 4, 7] with cost 14 in 0.002 seconds
GBFS h2 found solution for [0, 2, 1, 4, 3, 6, 5, 7] with cost 30 in 0.00997 seconds
GBFS h2 found solution for [4, 6, 5, 2, 1, 3, 7, 0] with cost 20 in 0.00199 seconds
GBFS h2 found solution for [6, 4, 3, 2, 1, 7, 0, 5] with cost 22 in 0.00199 seconds
GBFS h2 found solution for [6, 7, 3, 2, 0, 1, 5, 4] with cost 13 in 0.00199 seconds
GBFS h2 found solution for [1, 3, 6, 4, 5, 7, 2, 0] with cost 29 in 0.00898 seconds
GBFS h2 found solution for [7, 6, 0, 1, 2, 4, 5, 3] with cost 23 in 0.00399 seconds
GBFS h2 found solution for [7, 5, 3, 2, 1, 0, 4, 6] with cost 25 in 0.00399 seconds
GBFS h2 found solution for [2, 0, 1, 4, 7, 5, 3, 6] with cost 17 in 0.00199 seconds
GBFS h2 found solution for [6, 4, 3, 1, 7, 5, 0, 2] with cost 34 in 0.00798 seconds
GBFS h2 found solution for [5, 4, 0, 3, 1, 2, 6, 7] with cost 9 in 0.002 seconds
GBFS h2 found solution for [0, 2, 1, 5, 3, 4, 7, 6] with cost 22 in 0.00299 secon

In [40]:
analyze(a_star, analysis_set, 2, 4, h1)

A* h1 found solution for [2, 0, 5, 6, 1, 3, 4, 7] with cost 13 in 0.05585 seconds
A* h1 found solution for [0, 2, 1, 4, 3, 6, 5, 7] with cost 14 in 0.1496 seconds
A* h1 found solution for [4, 6, 5, 2, 1, 3, 7, 0] with cost 14 in 0.12804 seconds
A* h1 found solution for [6, 4, 3, 2, 1, 7, 0, 5] with cost 13 in 0.03989 seconds
A* h1 found solution for [6, 7, 3, 2, 0, 1, 5, 4] with cost 13 in 0.02951 seconds
A* h1 found solution for [1, 3, 6, 4, 5, 7, 2, 0] with cost 14 in 0.16253 seconds
A* h1 found solution for [7, 6, 0, 1, 2, 4, 5, 3] with cost 15 in 0.3351 seconds
A* h1 found solution for [7, 5, 3, 2, 1, 0, 4, 6] with cost 16 in 1.92432 seconds
A* h1 found solution for [2, 0, 1, 4, 7, 5, 3, 6] with cost 13 in 0.04914 seconds
A* h1 found solution for [6, 4, 3, 1, 7, 5, 0, 2] with cost 16 in 1.50966 seconds
A* h1 found solution for [5, 4, 0, 3, 1, 2, 6, 7] with cost 9 in 0.00199 seconds
A* h1 found solution for [0, 2, 1, 5, 3, 4, 7, 6] with cost 13 in 0.04093 seconds
A* h1 found solutio

In [41]:
analyze(a_star, analysis_set, 2, 4, h2)

A* h2 found solution for [2, 0, 5, 6, 1, 3, 4, 7] with cost 13 in 0.01998 seconds
A* h2 found solution for [0, 2, 1, 4, 3, 6, 5, 7] with cost 14 in 0.02042 seconds
A* h2 found solution for [4, 6, 5, 2, 1, 3, 7, 0] with cost 14 in 0.01596 seconds
A* h2 found solution for [6, 4, 3, 2, 1, 7, 0, 5] with cost 13 in 0.00405 seconds
A* h2 found solution for [6, 7, 3, 2, 0, 1, 5, 4] with cost 13 in 0.00399 seconds
A* h2 found solution for [1, 3, 6, 4, 5, 7, 2, 0] with cost 14 in 0.05281 seconds
A* h2 found solution for [7, 6, 0, 1, 2, 4, 5, 3] with cost 15 in 0.02593 seconds
A* h2 found solution for [7, 5, 3, 2, 1, 0, 4, 6] with cost 16 in 0.10871 seconds
A* h2 found solution for [2, 0, 1, 4, 7, 5, 3, 6] with cost 13 in 0.00798 seconds
A* h2 found solution for [6, 4, 3, 1, 7, 5, 0, 2] with cost 16 in 0.02793 seconds
A* h2 found solution for [5, 4, 0, 3, 1, 2, 6, 7] with cost 9 in 0.00253 seconds
A* h2 found solution for [0, 2, 1, 5, 3, 4, 7, 6] with cost 13 in 0.01197 seconds
A* h2 found solut

## Scaling Up

In [43]:
# We experimented with the following scaled up versions of the puzzle for GBFS h2 and A* h2:
#
# A* h2: 3x3, 3x4, 4x4
#
# GBFS h2: 3x3, 3x4, 4x4, 5x5, 8x8

scaled_3x3 = random_puzzle_set(3, 3, 10)
scaled_3x4 = random_puzzle_set(3, 4, 10)
scaled_4x4 = random_puzzle_set(4, 4, 10)

In [44]:
analyze(gbfs, scaled_3x3, 3, 3, h2)
analyze(a_star, scaled_3x3, 3, 3, h2)

GBFS h2 found solution for [1, 5, 4, 3, 0, 8, 2, 7, 6] with cost 24 in 0.0101 seconds
GBFS h2 found solution for [5, 4, 1, 8, 0, 7, 2, 3, 6] with cost 10 in 0.001 seconds
GBFS h2 found solution for [7, 3, 0, 6, 8, 5, 1, 2, 4] with cost 24 in 0.01028 seconds
GBFS h2 found solution for [2, 6, 8, 7, 0, 1, 4, 5, 3] with cost 22 in 0.00691 seconds
GBFS h2 found solution for [0, 6, 3, 2, 7, 4, 5, 1, 8] with cost 20 in 0.00698 seconds
GBFS h2 found solution for [6, 3, 1, 5, 2, 7, 0, 8, 4] with cost 18 in 0.00297 seconds
GBFS h2 found solution for [3, 7, 8, 5, 1, 6, 2, 4, 0] with cost 16 in 0.00432 seconds
GBFS h2 found solution for [6, 0, 8, 4, 1, 3, 5, 2, 7] with cost 25 in 0.00404 seconds
GBFS h2 found solution for [2, 7, 3, 5, 8, 1, 0, 4, 6] with cost 14 in 0.00304 seconds
GBFS h2 found solution for [5, 8, 6, 0, 1, 4, 7, 2, 3] with cost 20 in 0.004 seconds
ANALYSIS OF GBFS WITH H2

Average length of solution path: 15.0
Total length of solution path: 150

Average length of search path: 21.9

In [46]:
analyze(gbfs, scaled_3x4, 3, 4, h2)
analyze(a_star, scaled_3x4, 3, 4, h2)

GBFS h2 found solution for [1, 0, 9, 8, 2, 11, 6, 7, 10, 3, 5, 4] with cost 44 in 0.02299 seconds
GBFS h2 found solution for [3, 11, 8, 0, 9, 7, 2, 1, 6, 10, 4, 5] with cost 49 in 0.03658 seconds
GBFS h2 found solution for [1, 3, 0, 7, 4, 8, 11, 2, 6, 5, 9, 10] with cost 46 in 0.06011 seconds
GBFS h2 found solution for [2, 0, 6, 1, 7, 8, 9, 10, 11, 3, 4, 5] with cost 55 in 0.03215 seconds
GBFS h2 found solution for [2, 11, 5, 3, 7, 8, 10, 9, 4, 0, 1, 6] with cost 45 in 0.01806 seconds
GBFS h2 found solution for [11, 1, 7, 9, 4, 2, 8, 6, 5, 3, 10, 0] with cost 58 in 0.07206 seconds
GBFS h2 found solution for [6, 2, 0, 3, 9, 7, 10, 1, 11, 4, 5, 8] with cost 39 in 0.02883 seconds
GBFS h2 found solution for [5, 11, 1, 10, 4, 9, 3, 6, 2, 0, 7, 8] with cost 50 in 0.07023 seconds
GBFS h2 found solution for [4, 3, 8, 6, 7, 9, 11, 10, 2, 5, 1, 0] with cost 40 in 0.10488 seconds
GBFS h2 found solution for [6, 7, 8, 9, 11, 0, 2, 3, 5, 10, 1, 4] with cost 53 in 0.07092 seconds
ANALYSIS OF GBFS WIT

In [62]:
analyze(gbfs, scaled_4x4, 4, 4, h2)
# analyze(a_star, scaled_4x4, 4, 4, h2)

A* h2  timed out after 60 seconds on puzzle [3, 10, 6, 9, 0, 11, 4, 1, 7, 14, 5, 12, 2, 13, 15, 8]
A* h2 found solution for [1, 11, 2, 10, 14, 9, 4, 0, 12, 5, 8, 3, 15, 13, 7, 6] with cost 36 in 57.19664 seconds
A* h2 found solution for [3, 0, 13, 6, 8, 15, 14, 1, 5, 2, 9, 12, 7, 4, 11, 10] with cost 38 in 210.22741 seconds
A* h2 found solution for [5, 9, 1, 11, 4, 14, 15, 3, 2, 10, 12, 7, 6, 8, 0, 13] with cost 31 in 0.55503 seconds
A* h2 found solution for [15, 1, 6, 4, 8, 0, 13, 3, 11, 10, 5, 14, 7, 2, 9, 12] with cost 36 in 21.27337 seconds
A* h2  timed out after 60 seconds on puzzle [11, 1, 9, 6, 4, 0, 10, 5, 2, 15, 7, 14, 3, 13, 12, 8]
A* h2  timed out after 60 seconds on puzzle [0, 13, 1, 2, 3, 15, 4, 14, 6, 11, 9, 8, 10, 12, 5, 7]
A* h2 found solution for [4, 5, 2, 14, 13, 3, 10, 15, 1, 8, 6, 0, 9, 7, 11, 12] with cost 32 in 224.87245 seconds
A* h2  timed out after 60 seconds on puzzle [9, 3, 7, 6, 14, 13, 8, 15, 10, 5, 2, 11, 12, 4, 1, 0]
A* h2  timed out after 60 seconds on p

In [49]:
scaled_5x5 = random_puzzle_set(5, 5, 10)

In [53]:
scaled_8x8 = random_puzzle_set(8, 8, 10)

In [50]:
analyze(gbfs, scaled_5x5, 5, 5, h2)

GBFS h2 found solution for [22, 13, 14, 0, 4, 15, 8, 5, 19, 20, 23, 11, 17, 2, 3, 1, 9, 6, 12, 7, 18, 21, 16, 10, 24] with cost 226 in 1.2974 seconds
GBFS h2 found solution for [18, 22, 5, 17, 13, 21, 2, 4, 23, 24, 19, 11, 1, 7, 8, 12, 3, 6, 16, 14, 20, 0, 9, 15, 10] with cost 219 in 1.96838 seconds
GBFS h2 found solution for [5, 7, 20, 16, 22, 11, 21, 3, 10, 0, 9, 6, 13, 4, 12, 2, 8, 24, 1, 17, 15, 18, 14, 19, 23] with cost 202 in 3.28313 seconds
GBFS h2 found solution for [13, 6, 18, 9, 15, 7, 16, 17, 0, 8, 11, 2, 12, 14, 22, 4, 10, 24, 3, 1, 23, 20, 19, 21, 5] with cost 221 in 2.09126 seconds
GBFS h2 found solution for [17, 8, 18, 5, 22, 16, 9, 12, 24, 19, 20, 21, 23, 7, 0, 15, 14, 3, 6, 10, 4, 13, 1, 2, 11] with cost 234 in 2.42623 seconds
GBFS h2 found solution for [19, 17, 1, 15, 21, 9, 18, 13, 4, 23, 0, 6, 24, 7, 5, 10, 16, 22, 8, 11, 3, 20, 14, 2, 12] with cost 233 in 2.76604 seconds
GBFS h2 found solution for [24, 18, 1, 11, 10, 2, 6, 20, 4, 19, 7, 5, 8, 12, 23, 3, 15, 22, 13,

In [60]:
analyze(gbfs, scaled_8x8, 8, 8, h2, timeout=False)

GBFS h2 found solution for [15, 44, 29, 20, 9, 35, 33, 58, 6, 50, 0, 14, 38, 31, 26, 7, 52, 30, 40, 17, 1, 56, 27, 23, 13, 61, 63, 39, 22, 55, 32, 54, 36, 47, 25, 18, 45, 37, 5, 49, 19, 11, 42, 60, 24, 12, 46, 21, 4, 59, 53, 10, 51, 16, 41, 48, 28, 2, 62, 3, 43, 57, 8, 34] with cost 1085 in 212.52689 seconds
GBFS h2 found solution for [14, 30, 1, 45, 29, 20, 61, 49, 59, 7, 43, 11, 24, 17, 13, 2, 26, 37, 16, 25, 50, 47, 33, 44, 9, 62, 18, 48, 36, 38, 63, 6, 4, 27, 58, 42, 51, 15, 54, 60, 52, 41, 39, 55, 12, 23, 19, 34, 56, 22, 21, 8, 31, 10, 3, 28, 0, 35, 46, 57, 53, 5, 32, 40] with cost 1000 in 161.64545 seconds
GBFS h2 found solution for [29, 4, 5, 40, 28, 37, 9, 49, 60, 55, 47, 21, 31, 26, 30, 52, 57, 44, 8, 33, 10, 3, 50, 45, 42, 1, 54, 48, 11, 18, 19, 63, 2, 6, 23, 38, 39, 25, 27, 13, 12, 32, 36, 15, 7, 14, 58, 34, 56, 59, 20, 51, 17, 0, 46, 35, 24, 43, 22, 62, 61, 16, 41, 53] with cost 1046 in 233.88957 seconds
GBFS h2 found solution for [9, 36, 44, 5, 1, 32, 21, 6, 27, 59, 43, 12

## Archive

We decided to archive our original version of the UCS algorithm before optimizing it to show how we improved performance.

We were able to increase the speed of the algorithm by a factor of 20 by eliminating the need to iterate over a list to find/remove items.

In [ ]:
# def ucs(puzzle, m, n):
#     start = time.time()
#     opened = []
#     closed = []
#     seq = 0
    
#     heappush(opened, UCSNode(puzzle, seq=seq))
#     seq += 1

#     while opened:
#         if (time.time() - start > 60):
#             print('UCS timed out after 60 seconds on puzzle', puzzle)
#             return None, closed, 60
        
#         current = heappop(opened)
#         closed.append(current)
        
#         if goal(current.state, m, n):
#             t = round(time.time() - start, 5)
#             print('UCS found solution for', puzzle, 'with cost', current.cost, 'in', t, 'seconds')
#             return solution_path(current), closed, t
        
#         children = ucs_successor(current, m, n, seq)
#         for i in children:
#             seq += 1
#             in_opened = False
#             in_closed = False
            
#             for k in closed:
#                 if i.state == k.state:
#                     in_closed = True
#                     break

#             if in_closed:
#                 continue
                
#             for j in opened:
#                 if i.state == j.state:
#                     if i < j:
#                         opened.remove(j)
#                         heapify(opened)
#                         heappush(opened, i)
#                     in_opened = True
#                     break
    
#             if not in_opened:
#                 heappush(opened, i)
                
#     t = round(time.time() - start, 5)
#     print('UCS exhausted every path in', t, 'seconds without finding a solution')
#     return None, closed, t

# sol_path, search_path, t = ucs([3, 0, 1, 4, 2, 6, 5, 7], 2, 4)
# print(len(sol_path))
# print(len(search_path))

# for i in sol_path:
#     print(i.state, i.move, i.cost)

<hr>

## Demo

In [ ]:
# demo_set = get_puzzle_set('file name')

# solve_puzzle_set(demo_set, 2, 4, 'demo/output')